In [1]:
# Initial imports.
import pandas as pd
from path import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_HR =pd.read_csv("Resources/hr_R2.csv")
df_HR

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,21,50to99,Pvt Ltd,1,More,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50to99,Pvt Ltd,5,Between,0.0
2,11561,city_21,0.624,Other,No relevent experience,Full time course,Graduate,STEM,5,50to99,Pvt Ltd,0,Less,0.0
3,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,21,50to99,Funded Startup,4,More,0.0
4,21651,city_176,0.764,Other,Has relevent experience,Part time course,Graduate,STEM,11,50to99,Pvt Ltd,1,More,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18377,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,50to99,Pvt Ltd,1,More,1.0
18378,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50to99,Pvt Ltd,4,Between,1.0
18379,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,21,50to99,Pvt Ltd,4,More,0.0
18380,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,STEM,0,500to999,Pvt Ltd,2,Less,0.0


In [3]:
df_HR= df_HR.drop(["enrollee_id", "enrolled_university", "city"], axis = 1)
df_HR

,city_development_index,gender,relevent_experience,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,0.920,Male,Has relevent experience,Graduate,STEM,21,50to99,Pvt Ltd,1,More,1.0
1,0.776,Male,No relevent experience,Graduate,STEM,15,50to99,Pvt Ltd,5,Between,0.0
2,0.624,Other,No relevent experience,Graduate,STEM,5,50to99,Pvt Ltd,0,Less,0.0
3,0.767,Male,Has relevent experience,Masters,STEM,21,50to99,Funded Startup,4,More,0.0
4,0.764,Other,Has relevent experience,Graduate,STEM,11,50to99,Pvt Ltd,1,More,1.0
...,...,...,...,...,...,...,...,...,...,...,...
18377,0.878,Male,No relevent experience,Graduate,Humanities,14,50to99,Pvt Ltd,1,More,1.0
18378,0.920,Male,Has relevent experience,Graduate,STEM,14,50to99,Pvt Ltd,4,Between,1.0
18379,0.920,Male,Has relevent experience,Graduate,STEM,21,50to99,Pvt Ltd,4,More,0.0
18380,0.802,Male,Has relevent experience,High School,STEM,0,500to999,Pvt Ltd,2,Less,0.0


# Encoding

In [4]:
# Generate our categorical variable list
hr_cat = df_HR.dtypes[df_HR.dtypes == "object"].index.tolist()
hr_cat

['gender',
 'relevent_experience',
 'education_level',
 'major_discipline',
 'company_size',
 'company_type',
 'training_hours']

In [5]:
# Check the number of unique values in each column
df_HR[hr_cat].nunique()

gender                 3
relevent_experience    2
education_level        5
major_discipline       6
company_size           8
company_type           6
training_hours         3
dtype: int64

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_HR[hr_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(hr_cat)
encode_df.head()

,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,education_level_Graduate,education_level_High School,education_level_Masters,education_level_Phd,education_level_Primary School,...,company_size_less than10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,training_hours_Between,training_hours_Less,training_hours_More
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
# Merge one-hot encoded features and drop the originals
df_HR = df_HR.merge(encode_df,left_index=True, right_index=True)
df_HR = df_HR.drop(hr_cat,1)
df_HR

,city_development_index,experience,last_new_job,target,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,education_level_Graduate,...,company_size_less than10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,training_hours_Between,training_hours_Less,training_hours_More
0,0.920,21,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.776,15,5,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.624,5,0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.767,21,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.764,11,1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18377,0.878,14,1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
18378,0.920,14,4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
18379,0.920,21,4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
18380,0.802,0,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


# Split the Data into Training and Testing

In [8]:
# Define features set
X = df_HR.copy()
X = X.drop("target", axis=1)
X.head()

,city_development_index,experience,last_new_job,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,education_level_Graduate,education_level_High School,...,company_size_less than10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,training_hours_Between,training_hours_Less,training_hours_More
0,0.920,21,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.776,15,5,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.624,5,0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.767,21,4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.764,11,1,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [9]:
# Define target vector
y = df_HR["target"].values.reshape(-1, 1)
y[:5]

array([[1.],
       [0.],
       [0.],
       [0.],
       [1.]])

In [10]:
# Splitting into Train and Test sets

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=78, 
                                                    stratify=y)
X_train.shape

(13786, 36)

# SVM Model

In [13]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
svm = SVC(kernel='linear')                                      

In [14]:
# Fit the data
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [16]:
# Make predictions using the test data
y_pred = svm.predict(X_test)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test.flatten()
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [17]:
# Calculating the accuracy score.
accuracy_score(y_test, y_pred)

0.7545691906005222

In [18]:
# Displaying results
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      1.00      0.86      3468
         1.0       0.00      0.00      0.00      1128

    accuracy                           0.75      4596
   macro avg       0.38      0.50      0.43      4596
weighted avg       0.57      0.75      0.65      4596

